## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks

to use agents, we require 3 things
- A base LLM
- A tool that we will be interacting with
- An agent to control the interaction

#### In the following example, we will use the Math tool to perform some simple math operations

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI
import os
load_dotenv()

In [23]:
llm = GoogleGenerativeAI(
  model="gemini-1.5-pro-latest", 
  google_api_key=os.getenv("GOOGLE_API_KEY"), 
)

In [24]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do with.
from langchain.agents import Tool

In [25]:
# initialize the math tool

llm_math = LLMMathChain.from_llm(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

In [ ]:
print(tools[0].name, "---", tools[0].description)

### Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation mode that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [27]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="chat-zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

In [ ]:
zero_shot_agent("What is root over 25?")

In [ ]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)

### Using mutiple tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool We will then give the Agent a problem that initiates a search first followed by Calculation.

In [ ]:
# Define the Tools
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
  func=search.run,
  name="Search",
  description="Useful when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
  func=llm_math_chain.run,
  name="Calculator",
  description="Useful for when you are asked to perform math calculations"
)

In [30]:
# Define the Agent

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

In [ ]:
# Run the Agent with the problem

agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and VND values""")

### Create a custom tool

In [32]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
  """
  use this tool to send a prompt and get a JSON returned 
  with three fields - Topic, Question_Details and Detailed_Response
  """
  json_prompt = PromptTemplate.from_template(
  """Return a JSON object with an `answer` key that answers the following question: {question}. 
      The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
  )

  model = GoogleGenerativeAI(
    model="gemini-1.5-pro-latest", 
    google_api_key=os.getenv("GOOGLE_API_KEY"), 
  )

  json_parser = JsonOutputParser()
  json_chain = json_prompt | model | json_parser
  x = json_chain.invoke({"question": question})
  return x

In [33]:
from langchain.chat_models import ChatGooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory


In [36]:
from langchain.agents import initialize_agent
from langchain_google_genai import GoogleGenerativeAI
import os



tools = [StructuredResponseTool]

# ensure llm is defined
llm = GoogleGenerativeAI(
    model="gemini-1.5-pro-latest", 
    google_api_key=os.getenv("GOOGLE_API_KEY"), 
)

# initialize agent with tools
zero_shot_agent = initialize_agent(
        agent="zero-shot-react-description",
        tools=tools,
        llm=llm,
        verbose=True,
        max_iterations=10
)

In [ ]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")